In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import data
from IPython.display import HTML
import myWidgets
import logisticRegression
import numpy as np
from sklearn.metrics import confusion_matrix

df = data.get_data()

Le but de ce projet est de construire un modèle de prédiction simple afin de me familiariser avec les concepts de base d'intelligence artificielle. 

Le projet va devoir prédire quels clients sont plus susceptibles d'acheter une assurance caravane en fonction des données provenant du CoIL 2000 Challenge. 

In [ ]:
display(myWidgets.VarComparator(df.columns, df).get_widget())

In [119]:
discrete_vars, continuous_vars = data.get_var_by_types(df)
#corr = df.corr(numeric_only=True).loc[['CARAVAN']].sort_values(by='CARAVAN', axis=1, ascending=False).drop(columns='CARAVAN')
corr_continuous = df[continuous_vars].corr().loc[['CARAVAN']].drop(columns=['CARAVAN'])
corr_discrete = df[discrete_vars + ['CARAVAN']].corr(method='spearman').loc[['CARAVAN']].drop(columns=['CARAVAN'])

corr_combined = pd.concat([corr_continuous, corr_discrete], axis=1).sort_values(by='CARAVAN', axis=1, ascending=False)
corr_top = corr_combined.unstack().sort_values(ascending=False) #2d a 1d (A,B)
corr_combined = corr_combined.style.background_gradient(cmap='coolwarm', axis=None).format(precision=4)

display(HTML(corr_combined.to_html()))


,PPERSAUT,APERSAUT,APLEZIER,MKOOPKLA,PWAPART,PBRAND,PPLEZIER,MINKGEM,AWAPART,MOPLHOOG,MHKOOP,MRELGE,MAUT1,ABYSTAND,MBERHOOG,MSKA,PBYSTAND,ABRAND,MINK7512,MINK4575,MZPART,MBERMIDD,MOPLMIDD,PGEZONG,ALEVEN,MGEMOMV,AFIETS,AGEZONG,MGODPR,MFWEKIND,PWAOREG,MSKB1,PFIETS,AZEILPL,AWAOREG,MBERZELF,PLEVEN,MGODRK,AINBOED,PAANHANG,PZEILPL,AAANHANG,AMOTSCO,MFGEKIND,MAUT2,PMOTSCO,PINBOED,MGODOV,MSKB2,MGEMLEEF,AWABEDR,MINK3045,PWABEDR,MINK123M,PBESAUT,APERSONG,AVRAAUT,ABESAUT,PPERSONG,PVRAAUT,MAANTHUI,PTRACTOR,AWERKT,PWERKT,ATRACTOR,AWALAND,PWALAND,MRELSA,MGODGE,MSKC,MBERARBG,PBROM,ABROM,MFALLEEN,MBERBOER,MBERARBO,MOSTYPE,MZFONDS,MOSHOOFD,MRELOV,MSKD,MAUT0,MHHUUR,MINKM30,MOPLLAAG
CARAVAN,0.1510,0.1442,0.1057,0.0959,0.0953,0.0945,0.0914,0.0902,0.0894,0.0846,0.0785,0.0703,0.0700,0.0667,0.0651,0.0635,0.0628,0.0614,0.0579,0.0577,0.0575,0.0477,0.0435,0.0402,0.0371,0.0356,0.0341,0.0335,0.0329,0.0324,0.0305,0.0287,0.0287,0.0262,0.0223,0.0220,0.0211,0.0196,0.0180,0.0126,0.0117,0.0094,0.0086,0.0079,0.0077,0.0056,0.0055,0.0040,0.0040,0.0039,-0.0008,-0.0010,-0.0019,-0.0020,-0.0065,-0.0085,-0.0090,-0.0092,-0.0096,-0.0097,-0.0098,-0.0123,-0.0126,-0.0144,-0.0172,-0.0213,-0.0216,-0.0334,-0.0417,-0.0423,-0.0424,-0.0444,-0.0451,-0.0531,-0.0540,-0.0546,-0.0580,-0.0583,-0.0615,-0.0623,-0.0631,-0.0765,-0.0792,-0.0798,-0.0905


Les 10 variables les plus correlated avec la cible sont :

In [121]:
data.top_index_and_values(10 ,corr_top)

Contribution car policies                          PPERSAUT   - 0.1510
Number of car policies                             APERSAUT   - 0.1442
Number of boat policies                            APLEZIER   - 0.1057
Purchasing power class                             MKOOPKLA   - 0.0959
Contribution private third party insurance see L4  PWAPART    - 0.0953
Contribution fire policies                         PBRAND     - 0.0945
Contribution boat policies                         PPLEZIER   - 0.0914
Average income                                     MINKGEM    - 0.0902
Number of private third party insurance 1 - 12     AWAPART    - 0.0894
High level education                               MOPLHOOG   - 0.0846


In [124]:
X = df.drop('CARAVAN', axis=1) 
y = df['CARAVAN']  #targets
y_test_data = np.array(df['CARAVAN'].values)
unique, counts = np.unique(y_test_data, return_counts=True)

w,b = logisticRegression.logistic_regression(X, y)

Iteration 0: Cost = 0.6931471805599453
Iteration 100: Cost = 0.21512609583877254
Iteration 200: Cost = 0.21003823465275023
Iteration 300: Cost = 0.207351221555455
Iteration 400: Cost = 0.2057237895924097
Iteration 500: Cost = 0.20463711661572578
Iteration 600: Cost = 0.20385731333350487
Iteration 700: Cost = 0.20326687141200525
Iteration 800: Cost = 0.20280122864418634
Iteration 900: Cost = 0.20242217754648936


In [ ]:
y_prediction = logisticRegression.predict(X, w, b)
accuracy = np.mean(y_prediction == y_test_data)
print('Accuracy: ', accuracy)
conf_matrix = confusion_matrix(y_test_data, y_prediction)
print(conf_matrix)

np.savez("model_version/model_state_1.npz", w=w, b=b)

Accuracy:  0.9402164576533242
[[5473    0]
 [ 348    0]]
Accuracy:  0.9402164576533242
[[5473    0]
 [ 348    0]]


In [ ]:
# unique_values, counts = np.unique(y_prediction, return_counts=True)
# print("Unique Values:", unique_values)
# print("Counts:", counts)

y_prediction = logisticRegression.predict(X, w, b)


0.005274743885728184 0.3389342577136918 0.06043663050957348
[0 0 0 ... 0 0 0]
